<h1 align = center> <b> Reinforcement Learning (M2 MOSIG / MSIAM)<h1>

<h1 align = center>  Lab 2: Temporal Difference Learning: Q-Learning and SARSA <h1>


## Introduction

The purpose of this lab is to practice with *Temporal Difference Learning* method (i.e., *TD Learning*); including two methods: *Q-Learning* and *SARSA*.

We prepare the libraries needed for this lab by running the next 3 cells:

$
\newcommand{\vs}[1]{\mathbf{#1}} % vector symbol (\boldsymbol, \textbf or \vec)
\newcommand{\ms}[1]{\mathbf{#1}} % matrix symbol (\boldsymbol, \textbf)
\newcommand{\prob}{\mathbb{P}}
\def\U{V}
\def\action{\vs{a}}       % action
\def\A{\mathcal{A}}        % TODO
\def\actionset{\mathcal{A}} %%%
\def\discount{\gamma}  % discount factor
\def\state{\vs{s}}         % state
\def\S{\mathcal{S}}         % TODO
\def\stateset{\mathcal{S}}  %%%
%
\def\E{\mathbb{E}}
%\newcommand{transition}{T(s,a,s')}
%\newcommand{transitionfunc}{\mathcal{T}^a_{ss'}}
\newcommand{transitionfunc}{P}
\newcommand{transitionfuncinst}{P(\nextstate|\state,\action)}
\newcommand{transitionfuncpi}{\mathcal{T}^{\pi_i(s)}_{ss'}}
\newcommand{rewardfunc}{r}
\newcommand{rewardfuncinst}{r(\state,\action,\nextstate)}
\newcommand{rewardfuncpi}{r(s,\pi_i(s),s')}
\newcommand{statespace}{\mathcal{S}}
\newcommand{statespaceterm}{\mathcal{S}^F}
\newcommand{statespacefull}{\mathcal{S^+}}
\newcommand{actionspace}{\mathcal{A}}
\newcommand{reward}{R}
\newcommand{statet}{S}
\newcommand{actiont}{A}
\newcommand{newstatet}{S'}
\newcommand{nextstate}{\state'}
\newcommand{newactiont}{A'}
\newcommand{stepsize}{\alpha}
\newcommand{discount}{\gamma}
\newcommand{qtable}{Q}
\newcommand{finalstate}{\state_F}
%
\newcommand{\vs}[1]{\boldsymbol{#1}} % vector symbol (\boldsymbol, \textbf or \vec)
\newcommand{\ms}[1]{\boldsymbol{#1}} % matrix symbol (\boldsymbol, \textbf)
\def\vit{Value Iteration}
\def\pit{Policy Iteration}
\def\discount{\gamma}  % discount factor
\def\state{\vs{s}}         % state
\def\S{\mathcal{S}}         % TODO
\def\stateset{\mathcal{S}}  %%%
\def\cstateset{\mathcal{X}} %%%
\def\x{\vs{x}}                    % TODO cstate
\def\cstate{\vs{x}}               %%%
\def\policy{\pi}
\def\piparam{\vs{\theta}}         % TODO pparam
\def\action{\vs{a}}       % action
\def\A{\mathcal{A}}        % TODO
\def\actionset{\mathcal{A}} %%%
\def\caction{\vs{u}}       % action
\def\cactionset{\mathcal{U}} %%%
\def\decision{\vs{d}}       % decision
\def\randvar{\vs{\omega}}       %%%
\def\randset{\Omega}       %%%
\def\transition{T}       %%%
\def\immediatereward{r}    %%%
\def\strategichorizon{s}    %%% % TODO
\def\tacticalhorizon{k}    %%%  % TODO
\def\operationalhorizon{h}    %%%
\def\constalpha{a}    %%%
\def\U{V}              % utility function
\def\valuefunc{V}
\def\X{\mathcal{X}}
\def\meu{Maximum Expected Utility}
\def\finaltime{T}
\def\timeindex{t}
\def\iterationindex{i}
\def\decisionfunc{d}       % action
\def\mdp{\text{MDP}}
$

**Notice**: this notebook requires the following libraries: OpenAI *Gym*, NumPy, Pandas and Seaborn. There are two options:

  * If you want to run this notebook in your local machine, you can install them with the following command (and ignore the next cell):

``
pip install gym numpy pandas seaborn
``

  * Alternatively, if you want to run this notebook on Google Colab, just run the next cell. 

In [ ]:
colab_requirements = ["gym", "numpy","pandas","seaborn"]
import sys, subprocess
def run_subprocess_command(cmd):
    # run the command
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE)
    # print the output
    for line in process.stdout:
        print(line.decode().strip())
        
if "google.colab" in sys.modules:
    for i in colab_requirements:
        run_subprocess_command("pip install " + i)

In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import math
import gym
import numpy as np
import copy
import pandas as pd
import seaborn as sns
sns.set_context("talk")

## Setup the FrozenLake toy problem with OpenAI Gym

Similar to Lab1, we will use the FrozenLake toy problem as a testbed for our algorithms. A quick remind of FrozenLake is as follows (for more details, please refer to Lab 1's notebooks):

In [ ]:
## Load the FrozenLake environment, its states and actions:
env = gym.make('FrozenLake-v0')
# env = gym.make('FrozenLake-v1') ## Uncommentize this if you install the latest version of GymAI on your computer
states = list(range(env.observation_space.n))
actions = list(range(env.action_space.n))
is_final_array = np.full(shape=len(states), fill_value=np.nan, dtype=np.bool)
reward_array = np.full(shape=len(states), fill_value=np.NINF)                # np.NINF = negative infinity
transition_array = np.zeros(shape=(len(states), len(actions), len(states)))
for state in states:
    for action in actions:
        for next_state_tuple in env.P[state][action]:              # env.P[state][action] contains the next states list (a list of tuples)
            transition_probability, next_state, next_state_reward, next_state_is_final = next_state_tuple
            is_final_array[next_state] = next_state_is_final
            reward_array[next_state] = max(reward_array[next_state], next_state_reward)   # workaround: when we already are in state 15, reward is 0 if we stay in state 15 (in practice this never append as the simulation stop when we arrive in state 15 as any other terminal state)
            transition_array[state, action, next_state] += transition_probability
action_labels = { 0: "Move Left",   1: "Move Down",    2: "Move Right",    3: "Move Up"}
# For visualizing the environment
try: env.render()
except:pass # render not available
print('\n', 'states: ', states)
print('action name convetion', action_labels)
print('Reward:' ,reward_array)
print('Is final state:', is_final_array)



SFFF
FHFH
FFFH
HFFG

 states:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
action name convetion {0: 'Move Left', 1: 'Move Down', 2: 'Move Right', 3: 'Move Up'}
Reward: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Is final state: [False False False False False  True False  True False False False  True
  True False False  True]



<img src="https://raw.githubusercontent.com/dongquan-vu/teaching/main/FrozenLake_trans.PNG" style="float: left; width: 90%" />

#### **Important Remark:**
* In Lab1, we work with **model-based** methods: the learning agent "knows" the transition probabilities and the rewards; more precisely, it can "*ask for expectation*".

* **In this lab, we work with model-free** methods: this time, we assume that the learning agent *does not know* the world set-up; it can only "*ask for a sample of the next state and reward*". 

#### Extra functions for displaying results:

In the next cell, we write two function for displaying the states and Qtables in the FrozenLake environment.

In [ ]:
def qtable_display(q_array, title=None, figsize=(4,4), annot=True, fmt="0.1f", linewidths=.5, square=True, cbar=False, cmap="Reds"):
    num_actions = q_array.shape[1]
    global_figsize = list(figsize)
    global_figsize[0] *= num_actions
    fig, ax_list = plt.subplots(ncols=num_actions, figsize=global_figsize)   # Sample figsize in inches
    for action_index in range(num_actions):
        ax = ax_list[action_index]
        state_seq = q_array.T[action_index]
        states_display(state_seq, title=None, figsize=figsize, annot=True, fmt="0.1f", linewidths=.5, square=True, cbar=False, cmap="Reds", ax=ax)
    plt.suptitle(title)
    plt.show()

In [ ]:
def states_display(state_seq, title=None, figsize=(5,5), annot=True, fmt="0.1f", linewidths=.5, square=True, cbar=False, cmap="Reds", ax=None):
    size = int(math.sqrt(len(state_seq)))
    state_array = np.array(state_seq)
    state_array = state_array.reshape(size, size)
    if ax is None:
        fig, ax = plt.subplots(figsize=figsize)         # Sample figsize in inches
    sns.heatmap(state_array, annot=annot, fmt=fmt, linewidths=linewidths, square=square, cbar=cbar, cmap=cmap, ax=ax)
    if title is not None:     plt.title(title)
    if ax is None: plt.show()
    else: return ax

# Section 1: Temporal Difference for Value Estimation

## TD-LEARNING algorithm:

Recall: In order to evaluate a policy $\pi$, in previous lab:
* Bellman Equation:
 $$
  \U_{\pi}(s) = \mathbb{E} \left[ \reward(S_{t+1}) + \discount \U_{\pi}(S_{t+1}) | S_t = \state, A_t \sim \pi(\state) \right]
  $$

* In Lab1: approximating $\U_{\pi}$ by interating (turning Bellman's equation into an update rule) 
    \begin{equation}
      \U_{t+1}(\state) =   \mathbb{E} \left[ \reward(S_{t+1}) + \discount \U_{k}(S_{t+1}) | S_t = \state, A_t \sim \pi(\state) \right] .
    \end{equation}

 <h4 align = center> <b> Key question: How to approximate Value Function using model-free methods?<h4>



* Cannot find $\mathbb{E}$, can we use a **sample** instead: $\U_{t}(s) = \overbrace{R(S_{t+1}) + \gamma \U_{k}(S_{t+1})}^{\textrm{get from sampling}}$?

 $\Rightarrow$ This is likely very noisy. 

* **The main idea of TD-Learning:** Take a small step---controled via parameter $\alpha$---by using the temporal difference:

\begin{equation}
      \U_{t+1}(S_{k+1}) =  \U_{k}(S_t) + \alpha \cdot \left[\underbrace{\overbrace{\reward(S_{t+1}) + \discount \U_{k}(S_{t+1})}^{\textrm{target}} - \U_{k}(S_t)}_{\textrm{called the TD error}} \right] .
\end{equation}
Use this to update the Value Approximation.

PSEUDO-CODE OF TD LEARNING:


**Input**:$\quad\quad$ policy $\policy$ (to be evaluated), step size $\stepsize \in (0,1]$

**Initialize** $\valuefunc(\state)=0, \forall \state \in \statespace$<br>

<b>FOR EACH</b> episode<br>
	$\quad$ $\statet \leftarrow \text{env.reset}()$<br>
	$\quad$ <b>DO the following <b>UNTIL</b> $\statet$ is final:</b> <br>
		$\quad\quad$ $\actiont \leftarrow \pi(\statet)$<br>
		$\quad\quad$ $\newstatet, \reward \leftarrow \text{env.step}(\actiont)$<br>
		$\quad\quad$ $\valuefunc(\statet) \leftarrow \valuefunc(\statet) + \stepsize \left[ \reward + \discount ~ \valuefunc(\newstatet) ~ - ~ \valuefunc(\statet) \right]$<br>
		$\quad\quad$ $\statet \leftarrow \newstatet$<br>
	$\quad$ 

In the next cell, the TD-learning algorithm is prepared:

**Notice**: New syntax of Gym (particularly, FrozenLake):
* ```s = env.reset()```: assign the starting state to variable ```s```
* ``` new_state, reward, is_final_state, info = env.step(A)```: sample the next state when doing action ```A```.

In [ ]:
DISPLAY_EVERY_N_EPISODES = 50
def td_learning(policy, env, alpha_init=0.1, gamma=0.95, num_episodes=1000, display=False,alpha_factor=0.95):
    num_states = env.observation_space.n
    v_array = np.zeros(num_states)   # Initial value function
    alpha = alpha_init
    
    for episode_index in range(num_episodes):
        # For displaying purpose
        if display and episode_index % DISPLAY_EVERY_N_EPISODES == 0:
            states_display(v_array, title="Value function (ep. {})".format(episode_index), cbar=True, cmap="Reds")
        else: print('.', end="")
        value_function_history.append(v_array.copy())
        alpha_history.append(alpha)
        # End of displaying purpose            

        
        state = env.reset()
        is_final_state = False

        while not is_final_state:
            action = policy[state]
            new_state, reward, is_final_state, info = env.step(action)
            ##################
            # TD-LEARNING UPDATE
            v_array[state] = v_array[state] + alpha * (reward + gamma * v_array[new_state] - v_array[state])
            
            state = new_state
    
    return v_array

## Exercise 1: **The role of step-size $\alpha$:**



Now, we aim to test the ```td_learning``` algorithm by evaluating a specific policy (given in the next cell). 

*Remark:* This policy is actually the greedy_policy obtained from our (near) optimal value function in Lab1 (note again that in Lab1, the rewards and transition probablities are given a priori). The *target* values corresponding to this policy is: 
<img src="https://raw.githubusercontent.com/dongquan-vu/teaching/main/Opt_Value_FrozenLake.PNG" style="float: left" width="400px"/>
 


**In the next cell,** we run the td_learning algorithm and display the value function that it learnt. Then, we plot out the evolution of the value at each state and the evolution of the stepsize $\alpha$.


In [ ]:
value_function_history = []
alpha_history = []
policy = [0, 3, 3, 3,
          0, 0, 0, 0,
          3, 1, 0, 0,
          0, 2, 1, 0]
env._max_episode_steps = 10000
# TD-Learning:
v_array = td_learning(policy, env, display=False)
env.close()
# Display the value function
states_display(v_array, title="Value function", cbar=True, cmap="Reds")
# Plot out the evolution of value at each state
df_v_hist = pd.DataFrame(value_function_history)
df_v_hist.plot(figsize=(8,4))
plt.title("V(s) w.r.t iteration")
plt.ylabel("V(s)")
plt.xlabel("iteration")
plt.legend(bbox_to_anchor=(1,1.2), ncol = 3);
plt.show()

# Plotting the evolution of step-size alpha
plt.loglog(alpha_history)
plt.title("Alpha w.r.t iteration")
plt.ylabel("Alpha")
plt.xlabel("iteration");
plt.show()

**QUESTION 1.1:** 

1/ What is your observation on the evolution of value approximation? 
Do you think this is a good estimation of the target values given above? 

2/ Go back to the previous cell, run the TD-learning algorithm with smaller/larger parameter ```alpha```. How do you think ```alpha``` affects the evolution of values at states?

3/ Go back to the cell defining ```td_learning``` algorithm and modify it such that the step-size alpha is a **decreasing sequence** as ```episode_index``` increases. Particularly, you can try several options as follows:
  * (a) ```alpha``` decreases linearly: at each ```episode_index```, set ```alpha =``` $\frac{a}{1 + b \cdot \textrm{episode_index}}$ where $a < 1$ and $b$ are your chosen parameters. 
  * (b) ```alpha``` decreases exponentially:  at each ```episode_index```, set ```alpha =``` $a^{\textrm{episode_index}} $ where $a < 1$ is your chosen parameter (hint: you should choose $a$ closed to 1). 

Draw your conclusions on: what happens if the step-size $\alpha$ decreases too fast? when it decreases too slow?

**ANSWERS TO QUESTION 1.1:**


Please type your answer here. 

# Section 2: Tabular RL & Policy improvement


## Preliminary functions:


In the next cell, two functions are coded for finding the nearly optimal policy based on the value estimated by td_learning. Particularly, we implement the *greedy* and the $\epsilon$-*greedy* policies that agents will used to explore the environment and update their QTable:

$Q^{\pi}(s, a) =$ expected value when playing action $a$ at state $s$. 

QTable = a matrix (size $(|\mathcal{S}| \times |\mathcal{A}|)$) storing the current estimated Q-value. 


$\displaystyle \policy^{Q^{\pi}}(\statet) := \text{greedy}(\statet, Q^{\pi}) = \arg\max_{\actiont \in \actionspace} Q^{\pi}(\statet, \actiont)$


$\policy^{Q^{\pi},\epsilon}(\statet) := \epsilon\text{-greedy}(\statet, Q^{\pi}) = $
randomly choose between $\underbrace{\text{greedy}(\statet, Q^{\pi})}_{\text{with probability } 1 - \epsilon}$
and $~~ \underbrace{\text{a random action}}_{\text{with probability } \epsilon}$

In [ ]:
def greedy_policy(state, q_array):
    action = np.argmax(q_array[state, :])
    return action
def epsilon_greedy_policy(state, q_array, epsilon):
    if np.random.rand() < epsilon:
        action = np.random.randint(q_array.shape[1])
    else: action = greedy_policy(state, q_array)
    return action

## Exercise 2: Implement the SARSA algorithm

To find the optimal policy (or a nearly optimal policy) for the FrozenLake-v0 problem, we will first use the SARSA algorithm. SARSA has two main elements

* 1) Policy improvement: choose an action via $\varepsilon$-greedy.

* 2) Policy evaluation: Use TD-update (with respect to action chosen  $\varepsilon$-greedy) for Q-value:
$$
Q^{\pi}_{t+1} (s_t , a_t) \leftarrow Q^{\pi}_t(s_t, a_t) + \alpha \left( r_t + \gamma Q^{\pi}_t(s_{t+1}, a_{t+1}) - Q^{\pi}_t(s_t, a_t) \right) ,
$$
$\Rightarrow$ To estimate Q-value of policy $\pi$ $\Rightarrow$ this is called **On-policy** method.

Particularly, a pseudo code of SARSA is as follow:

---
SARSA


<b>Algorithm parameter</b>:	$\quad\quad$ discount factor $\discount$, 	$\quad\quad$ step size $\stepsize \in (0,1]$, 	$\quad\quad$ small $\epsilon > 0$

**Initialize** $\qtable(\state, \action)=0, \forall \state \in \statespace$

<b>FOR EACH</b> episode<br>
	$\quad$ $\statet \leftarrow \text{env.reset}()$<br>
	$\quad$ $\actiont \leftarrow \epsilon\text{-greedy}(\statet, Q)$<br>
$\quad$ <b>DO the following <b>UNTIL</b> $\statet$ is final:</b> <br>		$\quad\quad$ $\reward, \newstatet \leftarrow \text{env.step}(\actiont)$<br>
		$\quad\quad$ $\newactiont \leftarrow \epsilon\text{-greedy}(\newstatet, Q)$<br>
		$\quad\quad$ $Q(\statet,\actiont) \leftarrow Q(\statet,\actiont) + \stepsize \left[ \underbrace{\reward + \discount ~ Q(\newstatet,\newactiont) ~ - ~ Q(\statet,\actiont)}_{\text{TD error}} \right]$<br>
		$\quad\quad$ $\statet \leftarrow \newstatet$<br>
		$\quad\quad$ $\actiont \leftarrow \newactiont$<br>

**QUESTION 2.1**: In the next cell, you need to fill in the blank to implement SARSA. 

In [ ]:
DISPLAY_EVERY_N_EPISODES = 50
def sarsa(env, alpha_init=0.1, gamma=0.99, epsilon=0.5, num_episodes=10000, display=False):
    num_states = env.observation_space.n
    num_actions = env.action_space.n
    # Initial Q table (a matrix size num_states x num_actions)
    q_array = np.zeros([num_states, num_actions])   
    accum_reward = 0
    alpha = alpha_init

    for episode_index in range(num_episodes):
        # FOR displaying results
        if display and episode_index % DISPLAY_EVERY_N_EPISODES == 0:
            qtable_display(q_array, title="Q table", cbar=True)
        else: print('.', end="")
        q_array_history.append(q_array.copy())
        alpha_history.append(alpha)
        reward_history_sarsa.append(accum_reward)
        ##################
        ### BEGIN EXERCISE ###

        ## Step 0: Write your update-rule of alpha here:
        alpha = #.......


        is_final_state = False

        state = env.reset()
        action = epsilon_greedy_policy(state, q_array, epsilon)

        while not is_final_state:
            # FILLING IN THE BLANK:
            #Step 1: sample a new_state and reward (and check if it is the final state) by env.step(action) 
            # (hint: similar syntax to this step of td_learning )

            # Step 2: Sample a new_action by  epsilon_greedy_policy with respect to the (q_array)
            new_action = #.........
            # Step 3: Update the q_array (at the state and action) via TD-update:
            q_array[state, action] = #..............
            ### END EXERCISE ###
         
            #Reassign state and action to new_state and new_action:
            state = new_state
            action = new_action

        accum_reward+= reward  # For comparison purpose in Exercise 3
    return q_array

**Task**: In the next cell, we run SARSA and assign its output to ```q_array```.

In [ ]:
q_array_history = []
alpha_history = []
reward_history_sarsa=[]
env._max_episode_steps = 1000
# Use SARSA to update Q_array:
q_array = sarsa(env, epsilon=0.5, display=False)
env.close()
#Display the Q_table:
qtable_display(q_array, title="Q Table", cbar=True)

### Evaluate Policy with Gym

Run the next cell to count the number of successful trials of SARSA on 1000 episodes.

**Note**: OpenAI considers the task is solved if you reach 76\% of success over the episodes.

In [ ]:
env = gym.make('FrozenLake-v0')
env._max_episode_steps = 2000

reward_list = []
NUM_EPISODES = 5000

for episode_index in range(NUM_EPISODES):
    state = env.reset()
    done = False
    while not done:
        action = greedy_policy(state, q_array)
        state, reward, done, info = env.step(action)
    reward_list.append(reward)
reward_df = pd.DataFrame(reward_list)
print('Average reward (which is equivalent to a "success rate" in the FrozenLake environment as the total rewards in this environment are either 0 or 1):', np.average(reward_df))
env.close()

### Exercise 3: Implement the QLearning algorithm

## Exercise 3: Implement the Q-LEARNING algorithm


SARSA is an on-policy method. In this exercise, we implement Q-learning: an **off-policy** method: Q-learning doesn't estimate the Q-function of its current policy but it estimates the value *the optimal one*.

To do so, it uses the following update rule:
$$
Q_{t+1}(S_t,A_t) \leftarrow Q_t(S_t, A_t) + \alpha \left( r_t + \gamma \underbrace{\max_a Q_t(S_{t+1}, a)}_{\textrm{different from SARSA}} - Q_t(S_t, A_t) \right) .
$$

Intuitively, Q-learning is "being greedy" and chooses the $\textrm{argmax}_a Q(S,a)$ action (instead of choosing the next action by current policy like SARSA).

---
QLearning


<b>Algorithm parameter</b>:
	$\quad\quad$ discount factor $\discount$,
	$\quad\quad$ step size $\stepsize \in (0,1]$ ,
	$\quad\quad$ small $\epsilon > 0$,
	
**Initialize** $\qtable(\state, \action) =0, ~~~ \forall \state \in \statespace, \action \in \actionspace(\state)$<br>

<b>FOR EACH</b> episode<br>
	$\quad$ $\statet \leftarrow \text{env.reset}()$<br>
	$\quad$ <b>DO the following  <b>UNTIL</b> $\statet$ is final:</b> <br>
		$\quad\quad$ $\actiont \leftarrow \epsilon\text{-greedy}(\statet, Q)$<br>
		$\quad\quad$ $\reward, \newstatet \leftarrow \text{env.step}(\actiont)$<br>
		$\quad\quad$ $Q(\statet,\actiont) \leftarrow Q(\statet,\actiont) + \stepsize \left[ \underbrace{\reward + \discount ~ \max_{\action} Q(\newstatet, \action) ~ - ~ Q(\statet,\actiont)}_{\text{TD error}} \right]$<br>
		$\quad\quad$ $\statet \leftarrow \newstatet$<br>
	

**QUESTION 3.1.** In the next cell, you need to fill in the blank to implement the Q-LEARNING algorithm:

In [ ]:
DISPLAY_EVERY_N_EPISODES = 50

def q_learning(env, alpha_init=0.1, gamma=0.99, epsilon=0.5, num_episodes=10000, display=False):
    num_states = env.observation_space.n
    num_actions = env.action_space.n
    q_array = np.zeros([num_states, num_actions])   # Initial Q table
    accum_reward = 0
    alpha = alpha_init

    for episode_index in range(num_episodes):
        # For displaying purpose:  
        if display and episode_index % DISPLAY_EVERY_N_EPISODES == 0:
            qtable_display(q_array, title="Q table", cbar=True)
        else: print('.', end="")
        q_array_history.append(q_array.copy())
        alpha_history.append(alpha)
        reward_history_qlearning.append(accum_reward)
        
        
        ### BEGIN EXERCISE ###

        # Step 0:Write your update-rule of stepsize alpha here
        alpha = #...........

        is_final_state = False
        state = env.reset()

        while not is_final_state:
            # We take action from epsilon-greedy INSIDE the while-loop (unlike SARSA doing this outside of while-loop)
            action = epsilon_greedy_policy(state, q_array, epsilon)
            # Filling in the blank:
            # Step 1: Sample new_state, reward via env.step(action):
 
 
            # Step 2: Find the max_action = max_a (q_array[new_state, a]). 
            # Hints: you can use the greedy_policy function defined above:         
            max_action = #............
            # Step 3: TD-Update with max_action:
            q_array[state, action] = #..............

            ### END EXERCISE ###



            # Re-assign new_state to state:
            state = new_state
        
        accum_reward += reward  # For comparison purpose in Bonus question
    return q_array

**Task**: In the next cell, we run Q_LEARNING and assign its output to ```q_array```.

In [ ]:
q_array_history = []
alpha_history = []
reward_history_qlearning=[]

env._max_episode_steps = 1000
q_array = q_learning(env, display=False)
env.close()
qtable_display(q_array, title="Q Table", cbar=True)

### Evaluate Policy with Gym

As a measure of performance, run the next cell to count the number of successful trials of Q-learning on 1000 episodes.

**Note**: OpenAI considers the task is solved if you reach 76\% of success over the episodes.

In [ ]:
environment = gym.make('FrozenLake-v0')
environment._max_episode_steps = 1000
reward_list = []
NUM_EPISODES = 5000
for episode_index in range(NUM_EPISODES):
    state = environment.reset()
    done = False
    #t = 0
    while not done:
        #action = epsilon_greedy_policy(state, q_array, epsilon)
        action = greedy_policy(state, q_array)
        state, reward, done, info = environment.step(action)
    reward_list.append(reward)
reward_df = pd.DataFrame(reward_list)
print('Average reward (which is equivalent to a "success rate" in the FrozenLake environment as the total rewards in this environment are either 0 or 1):', np.average(reward_df))
environment.close()

**Question**: Which algorithm you think performs better SARSA or Q-LEARNING (given the same number of training episodes and alpha update rule)? (To mitigate the randomness, we should check the success rate in a large number of episodes).

### Bonus question: Compare SARSA and Q-LEARNING in training:
In the next cell, we plot out the accumulated reward while **training** the algorithms. Which one do you think perform worse in the beginning of the training? Any explanation for this?

CONCLUSION:
* Use SARSA when we "care" about the performance while training
* Use Q-Learning when we "do not care" about the performance while training.





In [ ]:
f,ax = plt.subplots()
ax.plot(reward_history_qlearning)
ax.plot(reward_history_sarsa)
ax.legend(['q_learning', 'sarsa'],bbox_to_anchor=(1, 0.5))
ax.set_ylabel('accumulated reward')

**ANSWERS FOR BONUS QUESTION**:

Please write your answer here.